In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from sklearn.model_selection import train_test_split

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.0-rc1


In [2]:
from pymystem3 import Mystem
from tqdm import tqdm_notebook,tqdm_pandas,tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors

import nltk

from tf_metrics import tf_metrics
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [3]:
tqdm_pandas(tqdm)

## Загрузка данных


In [4]:
df_train = pd.read_csv('60k-classes-text-classification/train.csv',sep=',')
df_test = pd.read_csv('60k-classes-text-classification/test.csv',sep=',')

In [5]:
df_train

,index,text,labels
0,0,Astrology: I am a Capricorn Sun Cap moon and c...,0
1,1,"I'm a triple Capricorn (Sun, Moon and ascendan...",0
2,2,How can I be a good geologist?,1
3,3,What should I do to be a great geologist?,1
4,4,How do I read and find my YouTube comments?,2
5,5,How can I see all my Youtube comments?,2
6,6,What can make Physics easy to learn?,3
7,7,How can you make physics easy to learn?,3
8,8,What was your first sexual experience like?,4
9,9,What was your first sexual experience?,4


In [6]:
df_train[df_train['labels']!=-1]['labels'].value_counts().count()

60416

In [15]:
df_train['text'].apply(len).median()

48.0

In [7]:
def text_prep_tags(text):
    
    text = text.lower()
    result = []
    # Убираем лишние символы
    #text = re.sub(r'[;,]',r' ',text).strip()
    text = re.sub(r'[^\w\s\.]',r'',text).strip()   
    #text = [token.text for token in razdel.tokenize(text)]
    # Делаем лемматизацию       
#     text = [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace() and lemma not in sw_ru
#             and lemma.strip() not in ['.','..','...']]
    
    
    for item in mystem.analyze(text):
      #  print(item)
        token = None
        if item.get('analysis'):
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            #and lemma not in sw_ru
            if not lemma.isspace()  and lemma.strip() not in ['.','..','...'] and lemma not in sw_ru:
     
                token = f'{lemma}_{upos_map[pos]}'
        else:
            lem_text = item["text"]
            if not lem_text.isspace() and lem_text.strip() not in ['.','..','...'] and lem_text not in sw_ru:
            
                token = f'{lem_text}_UNKN'
            
        if token:    
            result.append(token)

    
    
    
    # Лемматизатор неправильно разбивает слова с дефисом, поэтому исправляем это
#     if '-' in text:
#         for l in range(len(text)):
#             if text[l] == '-':
#                 text[l] = f'{text[l-1]}-{text[l+1]}'
#                 text[l-1] = text[l+1] = text[l]
    
    return result

In [8]:
mystem = Mystem()

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dmitriy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.corpus import stopwords
sw_ru = set(stopwords.words('russian'))

In [10]:
upos_map = {'A':'ADJ','ADV':'ADV','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCON','INTJ':'INTJ','NONLEX':'X','NUM':'NUM','PART':'PART','PR':'ADP','S':'NOUN','SPRO':'PRON','UNKN':'X' ,'V':'VERB'}

In [12]:
# http://vectors.nlpl.eu/repository/11/185.zip
word2vec = KeyedVectors.load_word2vec_format('w2v/tayga_upos_skipgram_300_2_2019.bin', binary=True)

In [13]:
count_v = CountVectorizer(tokenizer=text_prep_tags)
count_v.fit(tqdm_notebook(pd.concat([df_train['text'],df_test['text']])))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function text_prep_tags at 0x7f5429300730>,
        vocabulary=None)

In [14]:
train_token_text = df_train['text'].progress_apply(text_prep_tags)
test_token_text = df_test['text'].progress_apply(text_prep_tags)

100%|██████████| 58788/58788 [00:04<00:00, 11760.04it/s]


In [15]:
train_ids = train_token_text.progress_apply(lambda x: list(map(lambda y: count_v.vocabulary_[y] +1 , x)))
test_ids = test_token_text.progress_apply(lambda x: list(map(lambda y: count_v.vocabulary_[y] +1 , x)))

100%|██████████| 58788/58788 [00:00<00:00, 249081.00it/s]


In [16]:
vocab_size = len(count_v.vocabulary_)
sentence_size = 60
embedding_size = 300


# первые индексы в словаре зарезервированы за специальными токенами 
# для паддинга, начала предложения, и для слов которые не вошли в словарь
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2



print("Pad sequences (samples x time)")
train_ids = sequence.pad_sequences(train_ids, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

print("x_train shape:", train_ids.shape)
test_ids = sequence.pad_sequences(test_ids, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

print("x_train shape:", test_ids.shape)



Pad sequences (samples x time)
x_train shape: (150832, 60)
x_train shape: (58788, 60)


In [17]:
train_ids

array([[ 5241, 22539,  3913, ...,     0,     0,     0],
       [22876,  2472, 46991, ...,     0,     0,     0],
       [22149,  8670, 22539, ...,     0,     0,     0],
       ...,
       [22149, 14736,  3987, ...,     0,     0,     0],
       [22149, 14685, 49656, ...,     0,     0,     0],
       [50055, 41721, 22539, ...,     0,     0,     0]], dtype=int32)

In [22]:
df_train_triplet = df_train.copy()
df_train_triplet['anchor'] = pd.Series(train_ids.tolist())
df_train_triplet.drop(['index','text'],inplace=True,axis=1)

df_train_triplet_classes = df_train_triplet[df_train_triplet['labels']!=-1]
df_train_triplet_1 = df_train_triplet[df_train_triplet['labels']==-1]

In [23]:
df_train_triplet_classes.shape

(120832, 2)

In [24]:
df_train_triplet_classes = df_train_triplet_classes.reset_index().sort_values(['labels','index'])
df_train_triplet_classes['positive'] = df_train_triplet_classes.reset_index().sort_values(['labels','index'],ascending=[True,False])['anchor'].values

In [26]:
df_train_triplet_classes = df_train_triplet_classes[df_train_triplet_classes['index']%2==0]

In [57]:
new_df_triplet = pd.DataFrame({})

In [ ]:
df_train_triplet_classes.sample(replace=True,random_state=20)

In [36]:
get_sample_1(0)

136547    [42776, 13304, 32595, 2472, 34534, 30295, 2773...
147930    [22149, 14685, 3525, 28083, 46305, 43644, 2369...
125199    [50055, 14843, 34406, 16867, 18562, 45799, 181...
Name: anchor, dtype: object

In [53]:
def get_sample_classes(label):
    df = df_train_triplet_classes[df_train_triplet_classes['labels']!=label]['anchor']
    return df.sample(12,replace=True).values
def get_sample_1():
    df = df_train_triplet_1['anchor']
    return df.sample(3,replace=True).values

In [60]:
for triplet in tqdm_notebook(df_train_triplet_classes.itertuples(index=False)):
    trip =  {'anchor': [triplet.anchor]*15, 'positive': [triplet.positive]*15, 'negative': 
            list(get_sample_classes(triplet.labels))+list(get_sample_1())}

    df = pd.DataFrame(trip)
    new_df_triplet = new_df_triplet.append(df)


In [73]:
new_df_triplet = new_df_triplet.reset_index(drop=True)

In [74]:
new_df_triplet.to_json('triplet_df')

In [65]:
new_df_triplet.shape

(906240, 3)

In [66]:

def input_fn(x, labels, params, is_training):
    dataset = tf.data.Dataset.from_tensor_slices((x, labels))

    if is_training:
        dataset = dataset.shuffle(buffer_size=params['buffer_size'])
        dataset = dataset.repeat(count=params['num_epochs'])

    dataset = dataset.batch(params['batch_size'])
    dataset = dataset.prefetch(buffer_size=2)
    return dataset

In [67]:
def initializer(shape=None, dtype=None, partition_info=None):    
    vocab_dict = count_v.vocabulary_
    embedding_matrix = np.random.uniform(-1, 1, size=(vocab_size+1, embedding_size))
    num_loaded = 0
    for w, i in vocab_dict.items():
        v = None
        try:
            v = word2vec[w]
        except KeyError: # не нашли такой токен в словаре
                pass
        if v is not None :
            embedding_matrix[i+1] = v
            num_loaded += 1
   
    embedding_matrix = embedding_matrix.astype(np.float32)
    embedding_matrix[0] = np.zeros(300)
    return embedding_matrix

In [ ]:
def model_fn(features, labels, mode, params):  
    
    # Compute predictions.
    net = params['net']
    
    encoded_features = {}  
    
    with tf.variable_scope('encoder'):
        encoded_features['anchor'] = net(features['anchor'])
    with tf.variable_scope('encoder', reuse=True):
        encoded_features['positive'] = net(features['positive'])
    with tf.variable_scope('encoder', reuse=True):
        encoded_features['negative'] = net(features['negative'])
    
    
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op')
    f1 = tf_metrics.f1(labels=labels,
                               predictions=predicted_classes,num_classes=3,average='micro')
    
    metrics = {'accuracy': accuracy,'f1':f1}
    tf.summary.scalar('accuracy', accuracy[1])
    
    # Compute evaluation
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
    
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [68]:
def cnn_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)

    conv = tf.layers.conv1d(
        inputs=inputs,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    print(conv.get_shape())
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    print(pool.get_shape())
    hidden = tf.layers.dense(inputs=pool, units=100, activation=tf.nn.relu)
    logits = tf.layers.dense(inputs=hidden, units=64)

    return logits

In [ ]:
X_train_ids,X_val_ids,y_train,y_val = train_test_split(train_ids,X_train['label'],test_size=0.3)

In [62]:
# Final save to csv
submit = pd.read_csv('60k-classes-text-classification/sample_submission.csv', encoding='utf-8',sep=',',index_col='index')
submit['labels'] = -1
submit.to_csv('random.csv', encoding='utf-8',sep=',')